# World On Rails Baseline Model 

This baseline model is a random agent that applies constant throttle and random steering. It reads data and applies a policy.

### Data Reading

In [1]:
include("../data/data.jl")
using .Data

In [2]:
DATA_ROOT = "/home/enes/avg/WoR/dataset"
CONFIG_PATH = "/home/enes/avg/WoR/WorldOnRails/config.yaml"

"/home/enes/avg/WoR/WorldOnRails/config.yaml"

In [3]:
ds = Data.MainDataset(DATA_ROOT, CONFIG_PATH)


/home/enes/avg/WoR/dataset: 1877 frames x 3

Main.Data.MainDataset(Dict{Any, Any}("ego_model_dir" => "/kuacc/users/merciyes18/WorldOnRails/ego_model.th", "num_plan" => 5, "camera_yaws" => [0, -30, 30], "multi_cam" => true, "num_orient" => 5, "num_throts" => 3, "all_speeds" => true, "temperature" => 0.01, "ego_data_dir" => ["PATH TO PHASE 0 DATA"], "min_speeds" => 0.0…), 1877, Dict{Any, Any}(4986 => LMDB.Transaction(Ptr{Nothing} @0x00000000016fdf50), 4700 => LMDB.Transaction(Ptr{Nothing} @0x00000000037c2e10), 4576 => LMDB.Transaction(Ptr{Nothing} @0x00000000037c2e10), 2288 => LMDB.Transaction(Ptr{Nothing} @0x000000000387c1d0), 1703 => LMDB.Transaction(Ptr{Nothing} @0x0000000003899aa0), 1956 => LMDB.Transaction(Ptr{Nothing} @0x0000000003f74530), 2350 => LMDB.Transaction(Ptr{Nothing} @0x000000000387c1d0), 3406 => LMDB.Transaction(Ptr{Nothing} @0x000000000303d450), 2841 => LMDB.Transaction(Ptr{Nothing} @0x000000000363fcc0), 2876 => LMDB.Transaction(Ptr{Nothing} @0x000000000363fcc0)…), Dict{Any, Any}(4986 => LMDB.DBI(0x00000001, ""), 

In [5]:
@time data = Data.get_item(ds, 1870)

  0.009906 seconds (3.99 k allocations: 5.175 MiB)


(FixedPointNumbers.N0f8[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.N0f8[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.N0f8[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

...

FixedPointNumbers.N0f8[0.0N0f8 0.0N0f8 … 0.063N0f8 0.075N0f8; 0.0N0f8 0.0N0f8 … 0.059N0f8 0.071N0f8; 0.0N0f8 0.0N0f8 … 0.039N0f8 0.051N0f8]

FixedPointNumbers.N0f8[0.0N0f8 0.0N0f8 … 0.059N0f8 0.055N0f8; 0.0N0f8 0.0N0f8 … 0.055N0f8 0.051N0f8; 0.0N0f8 0.0N0f8 … 0.035N0f8 0.031N0f8]

FixedPointNumbers.N0f8[0.0N0f8 0.0N0f8 … 0.071N0f8 0.055N0f8; 0.0N0f8 0.0N0f8 … 0.067N0f8 0.051N0f8; 0.0N0f8 0.0N0f8 … 0.047N0f8 0.031N0f8], FixedPointNumbers.N0f8[0.42N0f8 0.275N0f8 … 0.0N0f8 0.0N0f8; 0.557N0f8 0.51N0f8 … 0.0N0f8 0.0N0f8; 0.137N0f8 0.706N0f8 … 0.557N0f8 0.557N0f8]

FixedPointNumbers.N0f8[0.42N0f8 0.42N0f8 

In [6]:
# visualize wide rgb semantic segmentation
data[2]

3×240×480 reinterpret(reshape, FixedPointNumbers.N0f8, ::Array{RGB{N0f8},2}) with eltype FixedPointNumbers.N0f8:
[:, :, 1] =
 0.42   0.275  0.275  0.275  0.275  …  0.0    0.0    0.0    0.0    0.0
 0.557  0.51   0.51   0.51   0.51      0.0    0.0    0.0    0.0    0.0
 0.137  0.706  0.706  0.706  0.706     0.557  0.557  0.557  0.557  0.557

[:, :, 2] =
 0.42   0.42   0.42   0.275  0.275  …  0.0    0.0    0.0    0.0    0.0
 0.557  0.557  0.557  0.51   0.51      0.0    0.0    0.0    0.0    0.0
 0.137  0.137  0.137  0.706  0.706     0.557  0.557  0.557  0.557  0.557

[:, :, 3] =
 0.275  0.42   0.42   0.42   0.275  …  0.0    0.0    0.0    0.0    0.0
 0.51   0.557  0.557  0.557  0.51      0.0    0.0    0.0    0.0    0.0
 0.706  0.137  0.137  0.137  0.706     0.557  0.557  0.557  0.557  0.557

...

[:, :, 478] =
 0.275  0.275  0.275  0.275  0.275  …  0.957  0.957  0.957  0.957  0.957
 0.51   0.51   0.51   0.51   0.51      0.137  0.137  0.137  0.137  0.137
 0.706  0.706  0.706  0.706  0.706    

In [7]:
# iterating over the dataset
x = () -> for data in ds end

#1 (generic function with 1 method)

In [8]:
@time x()

 38.307651 seconds (22.51 M allocations: 28.460 GiB, 1.86% gc time, 0.07% compilation time)


## Baseline Implementation

In [9]:
using Knet, Statistics, Random

In [10]:
# Defining the convolutional layer:
struct Conv; w; b; f; p; end
(c::Conv)(x) = c.f.(pool(conv4(c.w, dropout(x,c.p)) .+ c.b))
Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Conv

In [11]:
# Defining dense layer
struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [12]:
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)

In [69]:
baseline = Chain(Conv(5,5,3,20), Conv(5,5,20,50))

Chain((Conv(P(KnetArray{Float32, 4}(5,5,3,20)), P(KnetArray{Float32, 4}(1,1,20,1)), Knet.Ops20.relu, 0), Conv(P(KnetArray{Float32, 4}(5,5,20,50)), P(KnetArray{Float32, 4}(1,1,50,1)), Knet.Ops20.relu, 0)))

In [81]:
x = KnetArray(reshape(data[1], size(data[1])..., 1))

3×240×480×1 KnetArray{N0f8, 4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

...

[:, :, 478, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.078  0.059  0.051  0.063  0.075
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.075  0.055  0.047  0.059  0.071
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [76]:
colorview(RGB, x)

LoadError: MethodError: no method matching colorview(::Type{RGB}, ::KnetArray{Float32, 3})
[0mClosest candidates are:
[0m  colorview(::Type{C}, ::Any, [91m::Any[39m, [91m::Any...[39m) where C<:Colorant at /home/enes/.julia/packages/ImageCore/iXG0W/src/colorchannels.jl:186
[0m  colorview(::Type{C}) where C<:Colorant at /home/enes/.julia/packages/ImageCore/iXG0W/src/colorchannels.jl:215
[0m  colorview(::Type{C}, [91m::ImageMeta{T, N, A, P} where {A<:AbstractArray, P<:AbstractDict{Symbol, Any}}[39m) where {C<:Colorant, T, N} at /home/enes/.julia/packages/ImageMetadata/3vPdr/src/ImageMetadata.jl:253
[0m  ...

In [80]:
baseline(x)

LoadError: InexactError: check_top_bit(UInt64, -18880)